In [ ]:
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

root_dir = pathlib.Path('Beta')

img_height, img_width = 224, 224
batch_size = 32
validation_split = 0.1  
test_split = 0.1  


dataset = tf.keras.preprocessing.image_dataset_from_directory(
    root_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary'
)


class_names = dataset.class_names
print(f"Class names and their corresponding indices: {class_names}")


image_list, label_list = [], []
for images, labels in dataset:
    image_list.extend(images.numpy())
    label_list.extend(labels.numpy())


X_temp, X_test, y_temp, y_test = train_test_split(image_list, label_list, test_size=test_split, stratify=label_list)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=validation_split, stratify=y_temp)


train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)


all_labels = np.array(label_list).ravel() 
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(all_labels),
    y=all_labels
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print("Class weights:", class_weights_dict)


initial_lr = 1e-4  
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_lr,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001), input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Dropout(0.4),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    
    tf.keras.layers.Dense(1, activation='sigmoid') 
])


model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='binary_crossentropy',  
              metrics=['accuracy', tf.keras.metrics.AUC(name='AUC')])


history = model.fit(
    train_dataset,
    epochs=200,
    validation_data=val_dataset,
    class_weight=class_weights_dict,
    callbacks=[early_stopping]
)


test_loss, test_accuracy, test_auc = model.evaluate(test_dataset)
print(f"Test AUC: {test_auc}")
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


model.save('ImageClassifier_Beta.keras')


plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.show()
